In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.color as color
import random as r
import math
import cv2
import glob
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Input, concatenate, UpSampling2D
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [3]:
#original img size is 240*240
image_size = 192
smooth = 1 
num_of_aug = 1
num_epoch = 10
pul_seq = 'Flair'
sharp = False       # sharpen filter

label_num = 5   # 1 = necrosis, 2 = tumor core, 4 = ET, 5 = complete tumor

In [ ]:
def create_data(src, mask, label=False, image_size=192):
    files = glob.glob(src + mask, recursive=True)
    imgs = []
    print('Processing---', mask)
    for file in files:
        img = io.imread(file, plugin='simpleitk')
        if label:
            if label_num == 5:
                img[img != 0] = 1       #Region 1 => 1+2+3+4 complete tumor
            if label_num == 1:
                img[img != 1] = 0       #only left necrosis
            if label_num == 2:
                img[img == 2] = 0       #turn edema to 0
                img[img != 0] = 1       #only keep necrosis, ET, NET = Tumor core
            if label_num == 4:
                img[img != 4] = 0       #only left ET
                img[img == 4] = 1
            img = np.asarray(img, dtype = np.int32)
            img = img[50:90]
            for s in img:
                s = cv2.resize(s, (image_size, image_size), interpolation = cv2.INTER_NEAREST)
                imgs.append(s)
        else:
            img = ((img-np.min(img)) / np.max(img)) * 2 - 1
            img = np.asarray(img, dtype = np.float32)
            img = img[50:90]
            for s in img:
                s = cv2.resize(s, (image_size, image_size), interpolation = cv2.INTER_LINEAR)
                imgs.append(s)
#         for slice in range(50,90):     #choose the slice range
#             s = img[slice,:,:]
#             img_t = cv2.resize(s, (192, 192), interpolation = cv2.INTER_LINEAR)
#             img_t = np.expand_dims(img_t, axis=2)
#             img_t = np.expand_dims(img_t, axis=0)
#             img_g = augmentation(img_t, num_of_aug)
#             for n in range(img_g.shape[0]):
#                 imgs.append(img_g[n,:,:,:])
#         break
    name = './data/sample_{}_{}_mask'.format(pul_seq, image_size) if label else './data/sample_{}_{}_image'.format(pul_seq, image_size)
    np.save(name, np.array(imgs).astype('float32'))  # save at home
    print('Saved', len(files), 'to', name)

In [ ]:
def augmentation(scans, n):          #input img must be rank 4 
    datagen = ImageDataGenerator(
        featurewise_center=False,   
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=25,
        zoom_range=False)
    i = 0
    scans_g = scans.copy()
    for batch in datagen.flow(scans, batch_size=1, seed=1000): 
        scans_g = np.vstack([scans_g,batch])
        i += 1
        if i == n:
            break

    return scans_g

In [ ]:
def show_image(array, array2, random = False):
    n = 5
    if random == True:
        indices = np.random.randint(len(array), size=n)
        print('Show Random {} images and masks'.format(n))
    else:
        indices = np.arange(n)
        print('Show First {} images and masks'.format(n))
    images = array[indices, :]
    masks = array2[indices, :]
    fig, axes = plt.subplots(2, 5, figsize=(15,5))
    for i in range(5):
        image = images[i, :, :]
        mask = masks[i, :, :]
        axes[0, i].set_title("Image Index = {}".format(indices[i]))
        axes[0, i].axis("off")
        axes[0, i].imshow(image, cmap='gray')
        axes[1, i].set_title("Mask Index = {}".format(indices[i]))
        axes[1, i].axis("off")
        axes[1, i].imshow(mask, cmap='gray')
    plt.show()

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
def build_model(image_size=192):
    inputs = Input((image_size, image_size, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4= Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5= Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis = -1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6= Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    
    up7 = concatenate([UpSampling2D(size=(2, 2))(up6), conv3], axis = -1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7= Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    
    up8 = concatenate([UpSampling2D(size=(2, 2))(up7), conv2], axis = -1)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8= Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2, 2))(up8), conv1], axis = -1)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9= Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    
    conv10= Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)

    model.summary()
    
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
    return model

In [ ]:
# create_data('./BRATS2015_Training/HGG/', '**/*{}*.mha'.format(pul_seq), label=False, image_size=192)
# create_data('./BRATS2015_Training/HGG/', '**/*OT*.mha', label=True, image_size=192)

In [ ]:
x = np.load('./data/sample_{}_{}_image.npy'.format(pul_seq, image_size))
y = np.load('./data/sample_{}_{}_mask.npy'.format(pul_seq, image_size))

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
show_image(x, y, True)

In [ ]:
seed = 5
np.random.seed(seed)
np.random.shuffle(x)
np.random.seed(seed)
np.random.shuffle(y)

In [ ]:
show_image(x, y, True)

In [ ]:
# model = build_model(image_size)
x_train = np.expand_dims(x, -1)
y_train = np.expand_dims(y, -1)
# history = model.fit(x_train, y_train, batch_size=16, validation_split=0.2 ,epochs = num_epoch ,verbose=1, shuffle=True)
# model.save_weights('weights_{}_{}.h5'.format(image_size,num_epoch))

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(range(num_epoch), loss, marker='.', label='loss(training data)')
plt.plot(range(num_epoch), val_loss, marker='.', label='val_loss(evaluationdata)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
x_train.shape

In [ ]:
model = build_model(image_size)
model.load_weights("weights_192_10.h5")
pred = model.predict(x_train[0:10])

In [ ]:
x_train.shape

In [ ]:
pred.shape

In [ ]:
tmp = np.squeeze(pred)

In [ ]:
tmp.shape

In [ ]:
show_image(np.squeeze(pred), np.squeeze(y), False)